# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:

* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with Tensorflow (or TPUs)
* Exercises to try for yourself

## Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2023-05-12 10:04:05.127784: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 10:04:05.397648: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 10:04:05.399620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 10:04:06.768338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


In [2]:
# Create a tensor with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
print(f'Number of dimensions: {scalar.ndim}')

Number of dimensions: 0


In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimensions of our vector
print(f'Number of dimensions: {vector.ndim}')

Number of dimensions: 1


In [6]:
# Create a matrix (has more one dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])

matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# Check dimensions of our matrix
print(f'Number of dimensions: {matrix.ndim}')

Number of dimensions: 2


In [8]:
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16)  # specify data type with dtype parameter

another_matrix
# shape(row, columns)

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# Number of dimensions of another matrix
print(f'Number of dimensions: {another_matrix.ndim}')

Number of dimensions: 2


In [12]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                      [4,5,6]],
                     [[7,8,9],
                     [10,11,12]],
                     [[13, 14,15],
                     [16, 17,18]]])

tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [17]:
# Number of dimensions of our tensor
print(f'Number of dimensions: {tensor.ndim}')

Number of dimensions: 3


### What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

## Create tensors with tf.Variable()

In [19]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [20]:
# Create the same tensor with tf.Variable() as above
changable_tensor = tf.Variable([10, 7])
unchangable_tensor = tf.constant([10, 7])

In [21]:
changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [22]:
# Let's try change one of the elements in our changable tensor
changable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [23]:
changable_tensor[0] = 7

TypeError: 'ResourceVariable' object does not support item assignment

In [25]:
# How about we try .assing()
changable_tensor[0].assign(7)
changable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [26]:
# Let's try change our unchangeable tensor
unchangable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [27]:
unchangable_tensor[0] = 7

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [28]:
unchangable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

## Creating random tensors

Random tensors are tensors of some arbitrary size wich contain random nunmbers.

In [29]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)  # Set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [30]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [31]:
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [35]:
random_3 = tf.random.Generator.from_seed(7)
random_3 = random_3.normal(shape=(3, 2))

random_4 = tf.random.Generator.from_seed(7)
random_4 = random_4.normal(shape=(3, 2))

random_3, random_4, random_3 == random_4

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

## Shuffle the order of elements in a tensor

In [36]:
# Shuffle the tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2, 5]])

not_shuffled.ndim

2

In [37]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [38]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)  # Random shuffles a tensor along its FIRST DIMENSION

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [39]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [40]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [42]:
tf.random.set_seed(42)  # Global level seed
tf.random.shuffle(not_shuffled, seed=42)  # Operation level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

Isso é conhecido como um tensor de classificação 3 (3 dimensões), no entanto, um tensor pode ter uma quantidade arbitrária (ilimitada) de dimensões.

Por exemplo, você pode transformar uma série de imagens em tensores com forma (224, 224, 3, 32), onde:
* 224, 224 (as primeiras 2 dimensões) são a altura e a largura das imagens em pixels.
* 3 é o número de canais de cores da imagem (vermelho, verde, azul).
* 32 é o tamanho do lote (o número de imagens que uma rede neural vê a qualquer momento).

Todas as variáveis ​​acima que criamos são, na verdade, tensores. Mas você também pode ouvi-los referidos por seus nomes diferentes (os que demos a eles):
* **escalar**: um único número.
* **vetor**: um número com direção (por exemplo, velocidade do vento com direção).
* **matriz**: uma matriz bidimensional de números.
* **tensor**: uma matriz n-dimensional de números (onde n pode ser qualquer número, um tensor de dimensão 0 é um escalar, um tensor de dimensão 1 é um vetor).

Para aumentar a confusão, os termos matriz e tensor são frequentemente usados ​​de forma intercambiável.

Daqui para frente, já que estamos usando o TensorFlow, tudo o que nos referirmos e usarmos serão tensores.

Para saber mais sobre a diferença matemática entre escalares, vetores e matrizes, consulte a [postagem de álgebra visual de Math is Fun](https://www.mathsisfun.com/algebra/scalar-vector-matrix.html).

![diferença entre escalar, vetor, matriz, tensor](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/00-scalar-vector-matrix-tensor.png)

🛠 **Exercise:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [44]:
tf.random.set_seed(4029)

tensor1 = tf.random.uniform(shape=(2,2), seed=7) 
tensor2 = tf.random.uniform(shape=(2,2), seed=7)
tensor3 = tf.random.uniform(shape=(2,2), seed=7)
tensor4 = tf.random.uniform(shape=(2,2), seed=7)
tensor5 = tf.random.uniform(shape=(2,2), seed=7)

tensor1, tensor2, tensor3, tensor4, tensor5

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.17115712, 0.6531501 ],
        [0.8587817 , 0.874786  ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.7144613 , 0.37347782],
        [0.641343  , 0.10519004]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.2770716 , 0.5369843 ],
        [0.34588027, 0.60669184]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.5720482 , 0.18713129],
        [0.19723058, 0.462242  ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.2036438 , 0.44336116],
        [0.07702982, 0.71694434]], dtype=float32)>)

In [47]:
shuffled1 = tf.random.shuffle(tensor1)
shuffled2 = tf.random.shuffle(tensor2)
shuffled3 = tf.random.shuffle(tensor3)
shuffled4 = tf.random.shuffle(tensor4)
shuffled5 = tf.random.shuffle(tensor5)

shuffled1, shuffled2, shuffled3, shuffled4, shuffled5

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.8587817 , 0.874786  ],
        [0.17115712, 0.6531501 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.641343  , 0.10519004],
        [0.7144613 , 0.37347782]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.34588027, 0.60669184],
        [0.2770716 , 0.5369843 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.19723058, 0.462242  ],
        [0.5720482 , 0.18713129]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.07702982, 0.71694434],
        [0.2036438 , 0.44336116]], dtype=float32)>)

## Other ways to make tensors

In [48]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [49]:
# Create a tensor of all zeros
tf.zeros([3, 4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [50]:
import numpy as np

In [51]:
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

# X = tf.constant(some_matrix) -> Capital for Matrix
# y = tf.constant(vector) -> Non-Capital for Vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [52]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [53]:
A = tf.constant(numpy_A, shape=(2,3,4))
B = tf.constant(numpy_A)

A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [54]:
2 * 3 * 4

24

In [55]:
# shape=(2, 3, 4) -> 2 * 3 * 4 == len(numpy_A) -> Number of elements in numpy_A
print((2 * 3 * 4) == len(numpy_A))

True


In [56]:
C = tf.constant(numpy_A, shape=(2, 3, 5))

TypeError: Eager execution of tf.constant with unsupported shape. Tensor [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] (converted from [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]) has 24 elements, but got `shape` (2, 3, 5) with 30 elements).

In [ ]:
D = tf.constant(numpy_A, shape=(3, 8))